# Proyecto 1 - Analítica de textos 
## Análisis de sentimientos en películas

### Integrantes:
- Daniel Aguilera
- Vanessa Martínez
- Cristian Sánchez

### Descripción del problema
El objetivo de este notebook es realizar el desarrollo de la etapa 2 para el cual, se llevara acabo el analisis de sentimiento de las opiniones de las peliculas que se encuentran en el dataset MovieReviews.csv, para este fin se utilizara un modelo de redes neuronales para clasificar las opiniones en positivas y negativas. 

Se usara un modelo implementado usando redes neuronales debido a que este tipo de modelos son muy efectivos para el analisis de sentimientos, ya que son capaces de aprender patrones en los datos y asi poder clasificarlos de manera correcta.

## Referencias
1. [Datos a utilizar: MovieReviews.csv](./datos/MovieReviews.csv)
2. [Diccionario de datos: DiccionarioPeliculas.xlsx](./datos/DiccionarioPeliculas.xlsx)
3. [Datos de prueba: MovieReviewsPruebas.csv](./datos/MovieReviewsPruebas.csv)
4. [Datos procesados: MovieReviewsProcesado.csv](./datos/MovieReviewsProcesado.csv)
5. [Notebook de procesamiento y entendimiento de datos](./proy_desarrollo.ipynb)
6. [Información del negocio](./datos/enunciado.pdf)



In [8]:
# Realizamos los imports necesarios
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importamos MLP para crear la red neuronal
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

### Una vez importadas las librerias necesarias, se procede a cargar los datos preprocessados en el notebook anterior

In [9]:
# Cargamos el dataset
df_peliculas = pd.read_csv('./datos/MovieReviewsProcesado.csv')

# Mostramos las primeras filas
df_peliculas.head()

,Unnamed: 0,review_es,sentimiento,processed_text
0,0,Si está buscando una película de guerra típica...,1,si busc pelicul guerr tipic asi not aficion gu...
1,1,Supongo que algunos directores de películas de...,1,supon director pelicul luj sent busc abrig gra...
2,2,Es difícil contarle más sobre esta película si...,1,dificil contar el mas pelicul estropearlal dis...
3,3,"La película comienza muy lentamente, con el es...",1,pelicul comenz lent estil vid wallac napalm as...
4,4,Esta película es verdadera acción en su máxima...,1,pelicul verdader accion maxim expresion mejor ...
